In [1]:
# Python Library Imports
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from skimage import io
import joblib

Using TensorFlow backend.


In [ ]:
# Functions to extract data from csv file and load into a pandas dataframe 

def load_data(file):
    # columns of the data
    columns=['emotion','pixels','usage']
    # convert the csv data file into a dataframe 
    df=pd.read_csv(file,names=columns, na_filter=False)
    # preview dataframe to see if it was successful
    df.head(1)
    return df
def extract_features_and_labels(df):
    X=[] # features/pixels nested array of pixels
    Y=[] # Labels
    for index, row in df.iterrows():
        if(index!=0):
            
            Y.append(int(row['emotion']))
            X.append([int(p) for p in row['pixels'].split()]) # create array of pixel

    X, Y = np.array(X) / 255.0, np.array(Y)
    return X, Y
def cnn_model(num_class):
    model = Sequential()

    # 1 - Convolution
    model.add(Conv2D(64,(3,3), border_mode='same', input_shape=(48, 48,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 2nd Convolution layer
    model.add(Conv2D(128,(5,5), border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 3rd Convolution layer
    model.add(Conv2D(512,(3,3), border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 4th Convolution layer
    model.add(Conv2D(512,(3,3), border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))


    # Flattening
    model.add(Flatten())

    # Fully connected layer 1st layer
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))


    # Fully connected layer 2nd layer
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(num_class, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[categorical_accuracy])
    model.summary()
    return model

def emotion_analysis(emotions):
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    
    print("Predicted Emotion : ", objects[np.argmax(emotions)])
    print("Probability :",np.max(emotions))

def make_prediction(model):
    img = image.load_img('Data/disgust.jpeg', grayscale=True, target_size=(48, 48))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x /= 255

    custom = model.predict(x)
    print(custom[0])
    emotion_analysis(custom[0])
    
        
def ml_pipeline(file):
    # labels 
    labels = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    # load data 
    df = load_data(file)
    XY = extract_features_and_labels(df)
    X, Y = XY
    print("X shape", X.shape)
    print("Y shape", Y.shape)
    num_class = len(set(Y))
    print(num_class)
    # keras with tensorflow backend
    N, D = X.shape
    X = X.reshape(N, 48, 48, 1)
    
    #Split the data, 90% training and 10% testing
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
    y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
    y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)
    
    # create model architure 
    #model=cnn_model(num_class)
    K.tensorflow_backend.clear_session() # destroys the current graph and builds a new one
    model=cnn_model(num_class)
    K.set_value(model.optimizer.lr,1e-3) # set the learning rate
    
    fitted=model.fit(
            x=X_train,     
            y=y_train, 
            batch_size=128,
            epochs=124, 
            verbose=1, 
            validation_data=(X_test,y_test),
            shuffle=True
            )
    
    objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(objects))
    print(y_pos)
    
    #shape
    y_pred=model.predict(X_test)
    print(y_pred)
    y_test.shape
    
    make_prediction(model)
    #save trained model
    joblib.dump(model, "./production.joblib", compress=True)
    
ml_pipeline('Data/data.csv')
    

X shape (35887, 2304)
Y shape (35887,)
7


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(48, 48, 1..., padding="same")`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), padding="same")`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), padding="same")`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), padding="same")`


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 128)      

In [ ]:
import joblib
model = joblib.load("production.joblib")

In [ ]:
def emotion_analysis(emotions):
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    
    print("Predicted Emotion : ", objects[np.argmax(emotions)])
    print("Probability :",np.max(emotions))

def make_prediction(model):
    img = image.load_img('Data/disgust.jpeg', grayscale=True, target_size=(48, 48))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x /= 255

    custom = model.predict(x)
    print(custom[0])
    emotion_analysis(custom[0])

In [ ]:
make_prediction(model)